# Processing data for logistic regression

This notebook will transform data from a JSON file format submitted in the Kaggle competition Random Acts of Pizza and transform it into a dataframe that will include all features of itnerest to perform a logistic regression and predict who will get a pizza or not.

## Steps to cover

1. Data exploration
2. Feature selection
3. Data Wrangling
4. Output file

# Data exploration

Data is presented in two JSON file sets: train and test. The test data includes the outcome of interest "requester received pizza" plus a ton of other interesting information that we can use to predict. Let's look at the data.

In [132]:
# First let's import libraries of interest
import json
import pandas as pd

# Install Libraries
!pip install textblob
!pip install langdetect

# Import libraries - Sentiment
from textblob import TextBlob
import sys
import matplotlib.pyplot as plt
import numpy as np
import os
import nltk
nltk.download('vader_lexicon')
import re
import string
# from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Zaptetra\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [133]:
with open("../data/external/train.json") as f:
    data = json.load(f)
print(json.dumps(data[1], indent=4, sort_keys=True))

{
    "giver_username_if_known": "N/A",
    "number_of_downvotes_of_request_at_retrieval": 2,
    "number_of_upvotes_of_request_at_retrieval": 5,
    "post_was_edited": false,
    "request_id": "t3_rcb83",
    "request_number_of_comments_at_retrieval": 0,
    "request_text": "I spent the last money I had on gas today. Im broke until next Thursday :(",
    "request_text_edit_aware": "I spent the last money I had on gas today. Im broke until next Thursday :(",
    "request_title": "[Request] California, No cash and I could use some dinner",
    "requester_account_age_in_days_at_request": 501.11109953703703,
    "requester_account_age_in_days_at_retrieval": 1122.279837962963,
    "requester_days_since_first_post_on_raop_at_request": 0.0,
    "requester_days_since_first_post_on_raop_at_retrieval": 621.1270717592593,
    "requester_number_of_comments_at_request": 0,
    "requester_number_of_comments_at_retrieval": 1000,
    "requester_number_of_comments_in_raop_at_request": 0,
    "requeste

## What does it mean?

For those who are not familiar with Reddit and Random Acts of Pizza (I wasn't familiar either when I started the project), I present to you how this data looks like in their site. Not all the information appearing in here matches the information from the data, the reason is because there's more data available linked to the requester aside from the post itself. See figure 1 for reference.

![raop-example](./figures/01-raop-1.png)

Looking at the data, what stands out to me the most is:
- Upvotes/Downvotes
- Number of comments
- Time since publication
- Some say thanks, but are not tagged as "fulfilled"

Now let's look just to the single post to get the reference clearer.

![raop-example](./figures/01-raop-2.png)

Now inside the post we get to see some extra information, like the full title of the requester, the status (tagged as fulfilled) and the full post text. Additionally, the number of comments which can be read below the post that include the request status: who is willing to fulfill the request, how much money they are sending vs how much was actually sent, what pizza place, etc.

What stands out most:
- Title
- Text
- Upvotes
- Percentage of upvoted
- Number of comments
- Days since posted
- Tags (fulfilled, NSFW)

Now, we have access to the requester information:
- User subreddits
- User edited the post
- User flair

## Feature selection - Baseline model

In order to define the feature selection for our linear model we must choose features that are independent from each other and that we believe will be good predictors.

These are the features of interest after exploring the information available and looking the the actual post examples:

- Post upvotes
- Text length (word count)
- Text compound sentiment
- Title lenth (word count)
- Title sentiment
- Days since request
- User activity on redit (count of subreddits)
- Comment count after average request time
- User post's on redit at request (count)
- User posts's on raop (count)
- Account age (days)

The response variable will be true or false.

# Data Wrangling

The next step is process the input date to create a dataframe with the features of interest for the baseline model.

In [134]:
# Create a df from json

df = pd.json_normalize(data)
df.head()

giver_username_if_known  number_of_downvotes_of_request_at_retrieval  \
0                     N/A                                            0   
1                     N/A                                            2   
2                     N/A                                            0   
3                     N/A                                            0   
4                     N/A                                            6   

   number_of_upvotes_of_request_at_retrieval post_was_edited request_id  \
0                                          1           False   t3_l25d7   
1                                          5           False   t3_rcb83   
2                                          3           False   t3_lpu5j   
3                                          1            True   t3_mxvj3   
4                                          6           False  t3_1i6486   

   request_number_of_comments_at_retrieval  \
0                                        0   
1                                        0   
2                                        0   
3                                        4   
4                                        5   

                                        request_text  \
0  Hi I am in need of food for my 4 children we a...   
1  I spent the last money I had on gas today. Im ...   
2  My girlfriend decided it would be a good idea ...   
3  It's cold, I'n hungry, and to be completely ho...   
4  hey guys:\n I love this sub. I think it's grea...   

                             request_text_edit_aware  \
0  Hi I am in need of food for my 4 children we a...   
1  I spent the last money I had on gas today. Im ...   
2  My girlfriend decided it would be a good idea ...   
3  It's cold, I'n hungry, and to be completely ho...   
4  hey guys:\n I love this sub. I think it's grea...   

                                       request_title  \
0            Request Colorado Springs Help Us Please   
1  [Request] California, No cash and I could use ...   
2  [Request] Hungry couple in Dundee, Scotland wo...   
3  [Request] In Canada (Ontario), just got home f...   
4  [Request] Old friend coming to visit. Would LO...   

   requester_account_age_in_days_at_request  ...  requester_received_pizza  \
0                                  0.000000  ...                     False   
1                                501.111100  ...                     False   
2                                  0.000000  ...                     False   
3                                  6.518438  ...                     False   
4                                162.063252  ...                     False   

                     requester_subreddits_at_request  \
0                                                 []   
1  [AskReddit, Eve, IAmA, MontereyBay, RandomKind...   
2                                                 []   
3       [AskReddit, DJs, IAmA, Random_Acts_Of_Pizza]   
4  [GayBrosWeightLoss, RandomActsOfCookies, Rando...   

   requester_upvotes_minus_downvotes_at_request  \
0                                             0   
1                                            34   
2                                             0   
3                                            54   
4                                          1121   

   requester_upvotes_minus_downvotes_at_retrieval  \
0                                               1   
1                                            4258   
2                                               3   
3                                              59   
4                                            1225   

   requester_upvotes_plus_downvotes_at_request  \
0                                            0   
1                                          116   
2                                            0   
3                                           76   
4                                         1733   

   requester_upvotes_plus_downvotes_at_retrieval  requester_user_flair  \
0          

In [135]:
# let's check all the variables
df.columns

Index(['giver_username_if_known',
       'number_of_downvotes_of_request_at_retrieval',
       'number_of_upvotes_of_request_at_retrieval', 'post_was_edited',
       'request_id', 'request_number_of_comments_at_retrieval', 'request_text',
       'request_text_edit_aware', 'request_title',
       'requester_account_age_in_days_at_request',
       'requester_account_age_in_days_at_retrieval',
       'requester_days_since_first_post_on_raop_at_request',
       'requester_days_since_first_post_on_raop_at_retrieval',
       'requester_number_of_comments_at_request',
       'requester_number_of_comments_at_retrieval',
       'requester_number_of_comments_in_raop_at_request',
       'requester_number_of_comments_in_raop_at_retrieval',
       'requester_number_of_posts_at_request',
       'requester_number_of_posts_at_retrieval',
       'requester_number_of_posts_on_raop_at_request',
       'requester_number_of_posts_on_raop_at_retrieval',
       'requester_number_of_subreddits_at_request', 'r

## Data Transformation

Let's review the approapiate transformation for the variables of internest:

### Outcome variable - True False
**y** = requester_received_pizza


### Independent variables
**Post upvotes** = number_of_upvotes_of_request_at_retrieval - number_of_downvotes_of_request_at_retrieval

**Text length (word count)** = count(request_text)

**Text compound sentiment** = sentiment function (request_text)

**Title length (word count)** = count (request_title)

**Title sentiment** = sentiment function (request_title)

**Days since request** = requester_days_since_first_post_on_raop_at_retrieval - requester_days_since_first_post_on_raop_at_request

**User subs on (count of subreddits)** = requester_number_of_subreddits_at_request

**User activity comments** = requester_number_of_comments_at_retrieval - requester_number_of_comments_at_request

**User activity comments raop** = requester_number_of_comments_in_raop_at_retrieval - requester_number_of_comments_in_raop_at_request

**User posts reddit** = requester_number_of_posts_at_retrieval - requester_number_of_posts_at_request

**User posts raop** = requester_number_of_posts_on_raop_at_retrieval - requester_number_of_posts_on_raop_at_request

**Comment count** = request_number_of_comments_at_retrieval 

**User rate start** = requester_upvotes_plus_downvotes_at_request / requester_upvotes_plus_downvotes_at_request

**User rate end** = requester_upvotes_minus_downvotes_at_retrieval / requester_upvotes_plus_downvotes_at_retrieval

**Account age (days)** = requester_account_age_in_days_at_retrieval

Unused variables:
- post_was_edited
- request_id
- request_text_edit_aware
- unix_timestamp_of_request
- unix_timestamp_of_request_utc

Requester info unused vairables:
- requester_account_age_in_days_at_request
- requester_subreddits_at_request
- requester_upvotes_minus_downvotes_at_request
- requester_user_flair
- requester_username


In [136]:
# functions required

# Count words in a text
# Input: string
# Output: count of words
def count_words(string):
    word_list = string.split()
    num_words = len(word_list)
    return num_words

# test function
# count_words('I love chocolate and beef, please!')

# Provides compound sentiment of a text based on VADER Sentiment analysis
# Input: string
# Output: compound sentiment between -1 and 1 (Negative/Positive)
def sentiment_scores(sentence):
    
    # Create a sentiment analysis object
    sent = SentimentIntensityAnalyzer()
    sent_dict = sent.polarity_scores(sentence)
    
#     print("Pos: {}, Neu: {}, Neg: {}, Score: {}".format(sent_dict['pos'],sent_dict['neu'],
#                                                        sent_dict['neg'],sent_dict['compound']))
    
    if sent_dict['compound']>=0.2:
        sent_dict['class'] = 'Positive'
    elif sent_dict['compound']<= -0.2:
        sent_dict['class'] = 'Negative'
    else:
        sent_dict['class'] = 'Neutral'
    
#     return [sentence, sent_dict['pos'],
#                 sent_dict['neu'], sent_dict['neg'],
#                 sent_dict['compound'],sent_dict['class']]
    return sent_dict['compound']

# Check the function with an example
# sentiment_scores("70% of proud Georgians said they would ecstatically vote\
#                  and give their life for the great leader Trump in 2024.")

In [137]:
## Update dataframe with constructed variables

df['post_upvotes'] = df['number_of_upvotes_of_request_at_retrieval'] \
                     - df['number_of_downvotes_of_request_at_retrieval']
    
df['text_word_count'] = df['request_text'].apply(lambda x: count_words(x))
df['text_sentiment'] = df['request_text'].apply(lambda x: sentiment_scores(x))
df['title_word_count'] = df['request_title'].apply(lambda x: count_words(x))
df['title_sentiment'] = df['request_title'].apply(lambda x: sentiment_scores(x))
df['days_since_request'] = df['requester_days_since_first_post_on_raop_at_retrieval'] \
                           - df['requester_days_since_first_post_on_raop_at_request']
    
df['user_activity_comments'] = df['requester_number_of_comments_at_retrieval'] \
                               - df['requester_number_of_comments_at_request']
df['user_activity_comments_raop'] = df['requester_number_of_comments_in_raop_at_retrieval']\
                                    - df['requester_number_of_comments_in_raop_at_request']
    
df['user_posts_reddit'] = df['requester_number_of_posts_at_retrieval'] \
                          - df['requester_number_of_posts_at_request']

df['user_posts_raop'] = df['requester_number_of_posts_on_raop_at_retrieval'] \
                    - df['requester_number_of_posts_on_raop_at_request']

df['user_rate_start'] = df['requester_upvotes_plus_downvotes_at_request'] \
                        / df['requester_upvotes_plus_downvotes_at_request']

df['user_rate_end'] = df['requester_upvotes_minus_downvotes_at_retrieval'] \
                      / df['requester_upvotes_plus_downvotes_at_retrieval']

df.rename(columns={'requester_number_of_subreddits_at_request':'num_subreddits',
                  'request_number_of_comments_at_retrieval':'request_comment_counts',
                  'requester_account_age_in_days_at_retrieval':'account_age'})

df.drop(columns=['post_was_edited', 
                 'request_id',
                 'request_text_edit_aware',
                 'unix_timestamp_of_request',
                 'unix_timestamp_of_request_utc',
                 'requester_account_age_in_days_at_request',
                 'requester_subreddits_at_request',
                 'requester_upvotes_minus_downvotes_at_request',
                 'requester_user_flair',
                 'requester_username',
                 'giver_username_if_known',                 
                ], axis=1)
df.head()

giver_username_if_known  number_of_downvotes_of_request_at_retrieval  \
0                     N/A                                            0   
1                     N/A                                            2   
2                     N/A                                            0   
3                     N/A                                            0   
4                     N/A                                            6   

   number_of_upvotes_of_request_at_retrieval post_was_edited request_id  \
0                                          1           False   t3_l25d7   
1                                          5           False   t3_rcb83   
2                                          3           False   t3_lpu5j   
3                                          1            True   t3_mxvj3   
4                                          6           False  t3_1i6486   

   request_number_of_comments_at_retrieval  \
0                                        0   
1                                        0   
2                                        0   
3                                        4   
4                                        5   

                                        request_text  \
0  Hi I am in need of food for my 4 children we a...   
1  I spent the last money I had on gas today. Im ...   
2  My girlfriend decided it would be a good idea ...   
3  It's cold, I'n hungry, and to be completely ho...   
4  hey guys:\n I love this sub. I think it's grea...   

                             request_text_edit_aware  \
0  Hi I am in need of food for my 4 children we a...   
1  I spent the last money I had on gas today. Im ...   
2  My girlfriend decided it would be a good idea ...   
3  It's cold, I'n hungry, and to be completely ho...   
4  hey guys:\n I love this sub. I think it's grea...   

                                       request_title  \
0            Request Colorado Springs Help Us Please   
1  [Request] California, No cash and I could use ...   
2  [Request] Hungry couple in Dundee, Scotland wo...   
3  [Request] In Canada (Ontario), just got home f...   
4  [Request] Old friend coming to visit. Would LO...   

   requester_account_age_in_days_at_request  ...  text_sentiment  \
0                                  0.000000  ...          0.8323   
1                                501.111100  ...         -0.6908   
2                                  0.000000  ...          0.8074   
3                                  6.518438  ...          0.5154   
4                                162.063252  ...          0.9778   

   title_word_count  title_sentiment  days_since_request  \
0                 6           0.6124          792.420405   
1                10          -0.2960          621.127072   
2                10           0.6696          771.616181   
3                11           0.0000          734.517164   
4                14           0.8455          146.570567   

   user_activity_comments  user_activity_comments_raop  user_posts_reddit  \
0                       0                            0                  1   
1                    1000                            0                 11   
2                       0                            0                  1   
3                       5                            2                  1   
4                      38                            2                  2   

   user_posts_raop  user_rate_start  user_rate_end  
0                1              NaN       1.000000  
1                2              1.0       0.381268  
2                1              NaN       1.000000  
3                1              1.0       0.728395  
4                1              1.0       0.649179  

[5 rows x 44 columns]

In [196]:
## Reduce the dataset to the variabels of interest

df_2 = df.copy()

# Remove columns not used
df_2 = df_2.drop(columns=['giver_username_if_known',
       'number_of_downvotes_of_request_at_retrieval',
       'number_of_upvotes_of_request_at_retrieval', 
       'request_number_of_comments_at_retrieval', 
       'request_text_edit_aware', 
       'requester_account_age_in_days_at_request',
       'requester_account_age_in_days_at_retrieval',
       'requester_days_since_first_post_on_raop_at_request',
       'requester_days_since_first_post_on_raop_at_retrieval',
       'requester_number_of_comments_at_request',
       'requester_number_of_comments_at_retrieval',
       'requester_number_of_comments_in_raop_at_request',
       'requester_number_of_comments_in_raop_at_retrieval',
       'requester_number_of_posts_at_request',
       'requester_number_of_posts_at_retrieval',
       'requester_number_of_posts_on_raop_at_request',
       'requester_number_of_posts_on_raop_at_retrieval',
       'requester_number_of_subreddits_at_request',
       'requester_subreddits_at_request',
       'requester_upvotes_minus_downvotes_at_request',
       'requester_upvotes_minus_downvotes_at_retrieval',
       'requester_upvotes_plus_downvotes_at_request',
       'requester_upvotes_plus_downvotes_at_retrieval', 'requester_user_flair',
       'unix_timestamp_of_request','unix_timestamp_of_request_utc',
       'request_text','request_title','requester_username','request_id' ], axis=1)

# Clean values
# df_2['requester_received_pizza'] = df_2['requester_received_pizza'].apply(lambda x: 1 if (x==True) else 0 )
df_2['post_was_edited'] = df_2['post_was_edited'].apply(lambda x: 0 if (x==False) else 1)
df_2['user_rate_start'] = df_2['user_rate_start'].fillna(df_2['user_rate_start'].median())
df_2['user_rate_end'] = df_2['user_rate_end'].fillna(df_2['user_rate_end'].median())
df_2.insert(len(df_2.columns)-1, 'requester_received_pizza', df_2.pop('requester_received_pizza'))

# round data
df_2 = df_2.round(decimals=2)

df_2.head()

post_was_edited  post_upvotes  text_word_count  text_sentiment  \
0                0             1               67            0.83   
1                0             3               16           -0.69   
2                0             3               59            0.81   
3                1             1               30            0.52   
4                0             0              103            0.98   

   title_word_count  title_sentiment  days_since_request  \
0                 6             0.61              792.42   
1                10            -0.30              621.13   
2                10             0.67              771.62   
3                11             0.00              734.52   
4                14             0.85              146.57   

   user_activity_comments  user_activity_comments_raop  user_posts_reddit  \
0                       0                            0                  1   
1                    1000                            0                 11   
2                       0                            0                  1   
3                       5                            2                  1   
4                      38                            2                  2   

   user_posts_raop  user_rate_start  user_rate_end  requester_received_pizza  
0                1              1.0           1.00                     False  
1                2              1.0           0.38                     False  
2                1              1.0           1.00                     False  
3                1              1.0           0.73                     False  
4                1              1.0           0.65                     False

In [197]:
df_2.describe()

post_was_edited  post_upvotes  text_word_count  text_sentiment  \
count      4040.000000   4040.000000      4040.000000     4040.000000   
mean          0.159653      3.755941        77.529208        0.512564   
std           0.366330      9.048429        71.281563        0.519856   
min           0.000000     -7.000000         0.000000       -0.990000   
25%           0.000000      1.000000        35.000000        0.250000   
50%           0.000000      2.000000        59.000000        0.740000   
75%           0.000000      4.000000        97.000000        0.920000   
max           1.000000    314.000000       854.000000        1.000000   

       title_word_count  title_sentiment  days_since_request  \
count       4040.000000      4040.000000         4040.000000   
mean          12.422525         0.053941          502.576198   
std            6.965016         0.412741          270.740701   
min            1.000000        -0.930000            0.000000   
25%            8.000000        -0.250000          250.472500   
50%           11.000000         0.000000          506.305000   
75%           16.000000         0.400000          761.380000   
max           52.000000         0.970000         1025.410000   

       user_activity_comments  user_activity_comments_raop  user_posts_reddit  \
count             4040.000000                  4040.000000        4040.000000   
mean               174.327475                     1.981683          19.550248   
std                293.440622                     4.883963          47.282088   
min                  0.000000                     0.000000           0.000000   
25%                  2.000000                     0.000000           1.000000   
50%                 28.000000                     1.000000           4.000000   
75%                185.000000                     2.000000          18.000000   
max               1000.000000                   135.000000         998.000000   

       user_posts_raop  user_rate_start  user_rate_end  
count      4040.000000           4040.0    4040.000000  
mean          1.175495              1.0       0.492458  
std           0.481542              0.0       0.219908  
min           0.000000              1.0      -0.560000  
25%           1.000000              1.0       0.360000  
50%           1.000000              1.0       0.530000  
75%           1.000000              1.0       0.630000  
max           9.000000              1.0       1.000000

In [198]:
# Now save for future use
# Will save as csv in data/interim

df_2.to_csv('../data/interim/logit_sentiments.csv', index=False)